In [4]:
import os
import time

In [5]:
import requests

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

In [9]:
from pymongo import MongoClient

In [10]:
HOST_IP = 'localhost'
DB_PORT = 27017
client = MongoClient(HOST_IP, DB_PORT)
db = client.leetcode
collection = db.answers

In [26]:
def get_answer_urls(url):
    """ return at most 10 top voted answers
    """
    driver.get(url)
    time.sleep(3)
    dropdown_buttons = driver.find_elements_by_css_selector('button.dropdown-toggle')
    assert len(dropdown_buttons) == 2
    dropdown_buttons[1].click()
    time.sleep(2)
    driver.find_element_by_css_selector('a.most_votes').click()
    time.sleep(5)
    elements = driver.find_elements_by_css_selector('ul.topic-list h2.title > a')[:10]
    links = [ele.get_attribute("href") for ele in elements]
    return links

In [12]:
def get_answer(url):
    """ element.get_attribute('innerHTML')
    elem.get_attribute("outerHTML")
    """
    answer = {}
    driver.get(url)
    time.sleep(2)
    up_button = driver.find_element_by_css_selector('li i.fa-angle-double-up')
    up_button.click()
    driver.implicitly_wait(3)
    answer["url"] = url
    author_element = driver.find_element_by_css_selector("span.username a")
    answer["author"] = author_element.text
    answer["author_url"] = author_element.get_attribute('href')
    vote_element = driver.find_element_by_css_selector("div.vote-count")
    answer["vote"] = int(vote_element.text)
    content_element = driver.find_element_by_css_selector("div.answer .content")
    answer["content"] = content_element.get_attribute('outerHTML')
    question_element = driver.find_element_by_css_selector("a#back_to_leetcode")
    question_url = question_element.get_attribute('href')
    answer["question_url"] = question_url
    return answer

In [20]:
def get_real_url(url):
    url_list = url.split('/')
    try:
        num = int(url_list[-1])
        return "/".join(url_list[:-1])
    except:
        return url

In [27]:
# chrome_driver = "/home/eric/hub/personal/leetcode-best-answers/chromedriver"
chrome_driver = "/Users/Jing/Awesome/github/leetcode-best-answers/chromedriver_mac"
os.environ["webdriver.chrome.driver"] = chrome_driver
# display = Display(visible=0, size=(1800, 900))
# display.start()
driver = webdriver.Chrome(chrome_driver)
#driver.set_window_size(1280, 800)
driver.get("https://discuss.leetcode.com/category/8/oj")
time.sleep(2)
driver.find_element_by_link_text("Login").click()
time.sleep(2)
username = driver.find_element_by_id("id_login")
password = driver.find_element_by_id("id_password")
# your password!
username.send_keys("...@gmail.com")
password.send_keys("...")
driver.find_element_by_css_selector('button.btn-primary').click()
time.sleep(2)
driver.get("https://discuss.leetcode.com/category/8/oj")
time.sleep(2)
links = [link.get_attribute("href") for link in driver.find_elements_by_css_selector('ul.categories h2 a')]
print "There are", len(links), "problems"

with open('urls.txt', 'w') as urls:
    for link in reversed(links[-11:]):
        print "getting ...", link
        answer_urls = get_answer_urls(link)
        print len(answer_urls)
        for answer_url in answer_urls:
            real_answer_url = get_real_url(answer_url)
            urls.write(real_answer_url + "\n")
            # answer = get_answer(real_answer_url)
            # write answer into the database
time.sleep(5)
driver.quit()
# display.stop()

There are 377 problems
getting ... https://discuss.leetcode.com/category/497/combination-sum-iv
10
getting ... https://discuss.leetcode.com/category/496/wiggle-subsequence
10
getting ... https://discuss.leetcode.com/category/495/guess-number-higher-or-lower-ii
10
getting ... https://discuss.leetcode.com/category/492/guess-number-higher-or-lower
10
getting ... https://discuss.leetcode.com/category/491/find-k-pairs-with-smallest-sums
10
getting ... https://discuss.leetcode.com/category/490/super-pow
10
getting ... https://discuss.leetcode.com/category/455/sum-of-two-integers
10
getting ... https://discuss.leetcode.com/category/454/range-addition
10
getting ... https://discuss.leetcode.com/category/453/plus-one-linked-list
10
getting ... https://discuss.leetcode.com/category/452/largest-divisible-subset
10
getting ... https://discuss.leetcode.com/category/451/valid-perfect-square
10


In [28]:
!open urls.txt

In [ ]:
"""
# could be useful in the future
cookies = driver.get_cookies()
headers = {
"User-Agent":
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
}
s = requests.session()
s.headers.update(headers)

for cookie in driver.get_cookies():
    c = {cookie['name']: cookie['value']}
    s.cookies.update(c)
"""